In [1]:
import pandas as pd
import numpy as np 
import wget
import zipfile
import os
from datetime import date
import datetime

In [2]:
# Codes for municipalities
xcodes = pd.read_csv('Codes_municipality.csv',encoding='utf-8',low_memory=False)
xcodes = xcodes[(xcodes['CLAVE_MUNICIPIO']<997) & (xcodes['CLAVE_ENTIDAD']<36)].reset_index()
del xcodes['index']

In [3]:
xcodes

,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD
0,1,AGUASCALIENTES,1
1,2,ASIENTOS,1
2,3,CALVILLO,1
3,4,COSÍO,1
4,5,JESÚS MARÍA,1
5,6,PABELLÓN DE ARTEAGA,1
6,7,RINCÓN DE ROMOS,1
7,8,SAN JOSÉ DE GRACIA,1
8,9,TEPEZALÁ,1
9,10,EL LLANO,1


In [3]:
# Dates with historical data and breaks
end_date    = date(2021, 2, 11)
start_date  = date(2020, 4, 12)
break_date  = date(2020, 10, 6)
dateseries  = [date.fromordinal(i) for i in range(start_date.toordinal(), end_date.toordinal()+1)]

# Empty dataset
columns     = ['id','CLAVE_MUNICIPIO','CLAVE_ENTIDAD','DeathsR']
im = xcodes['CLAVE_MUNICIPIO'][0]
ie = xcodes['CLAVE_ENTIDAD'][0]
mxr         = pd.DataFrame(index=dateseries,columns=columns)
mxr         = mxr.reset_index()
mxr         = mxr.rename(columns={"index": "date"})
mxr['date'] = pd.to_datetime(mxr['date'])
mxr['id']              = 0
mxr['CLAVE_MUNICIPIO'] = im
mxr['CLAVE_ENTIDAD']   = ie
    
for j in range(1,len(xcodes)):
    print("Working on municipality code "+str(j)+" out of "+str(len(xcodes)-1), end="\r")
    im = xcodes['CLAVE_MUNICIPIO'][j]
    ie = xcodes['CLAVE_ENTIDAD'][j]
    mx         = pd.DataFrame(index=dateseries,columns=columns)
    mx         = mx.reset_index()
    mx         = mx.rename(columns={"index": "date"})
    mx['date'] = pd.to_datetime(mx['date'])
    mx['id']              = j
    mx['CLAVE_MUNICIPIO'] = im
    mx['CLAVE_ENTIDAD']   = ie
    mxr = mxr.append(mx)
    del mx,im,ie
    
mxr = mxr.reset_index()
del mxr['index']

In [8]:
# Download the entire raw data and build a new dataset with specific selections - as reported!!
# Data address: https://www.gob.mx/salud/documentos/datos-abiertos-152127
    
for i in dateseries:
    print('Working on: '+str(i), end="\r")

    # Extract year, month, day
    year = str(i.year-2000)
    if i.month<10:
        month = str(0)+str(i.month)
    else:
        month = str(i.month)
    if i.day<10:    
        day   = str(0)+str(i.day)
    else:   
        day   = str(i.day)

    # Download and extract file    
    filenamezip = 'datos_abiertos_covid19_'+day+'.'+month+'.20'+year+'.zip'
    if i.year<=2020:
        url = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/'+month+'/'+filenamezip
    else:
        url = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/20'+year+'/'+month+'/'+filenamezip
    filenamezip = wget.download(url)
    with zipfile.ZipFile(filenamezip, 'r') as zip_ref:
        zip_ref.extractall()

    # Read data and delete files
    filename = year+month+day+'COVID19MEXICO.csv'
    df = pd.read_csv(filename,encoding='latin1',low_memory=False)
    os.remove(filenamezip)
    os.remove(filename)

    # Municipality and entity
    for j in range(0,len(xcodes)):
        im = xcodes['CLAVE_MUNICIPIO'][j]
        ie = xcodes['CLAVE_ENTIDAD'][j]

        # Data selection
        if i<=break_date:
            s = df[
                (df['FECHA_DEF'] != '9999-99-99') & 
                (df['RESULTADO'] == 1) 
                &
                (df['MUNICIPIO_RES'] == im) &
                (df['ENTIDAD_RES'] == ie) 
            ][['FECHA_DEF','RESULTADO']] .groupby("FECHA_DEF") .count() .cumsum()
            s.index= pd.to_datetime(s.index)
            idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
            s = s.reindex(idx).fillna(method='ffill').fillna(0)
            s = s.rename(columns={'RESULTADO': 'DeathsR'})
        else:
            s = df[
                (df['FECHA_DEF'] != '9999-99-99') & 
                (df['CLASIFICACION_FINAL'] <= 3) 
                &
                (df['MUNICIPIO_RES'] == im) &
                (df['ENTIDAD_RES'] == ie)
            ][['FECHA_DEF','CLASIFICACION_FINAL']] .groupby("FECHA_DEF") .count() .cumsum()
            s.index= pd.to_datetime(s.index)
            idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
            s = s.reindex(idx).fillna(method='ffill').fillna(0)
            s = s.rename(columns={'CLASIFICACION_FINAL': 'DeathsR'})
        
        mxr.loc[(mxr['id']==j) & (mxr['date']==pd.to_datetime(i)),'DeathsR'] = s['DeathsR'][-1]
        #mxr.loc[(mxr['id']==j) & (mxr['date']==pd.to_datetime(i))]['DeathsR'] = s['DeathsR'][-1]

        del s
        del idx 

    del df

    

In [34]:
# Save dataset 
mxr.to_csv('Data_Reported_all_muni__mxr.csv', index=False)

#pd.set_option('display.max_rows', 500)
#mxr[(mxr['CLAVE_MUNICIPIO']==15) & (mxr['CLAVE_ENTIDAD']==9)]
#mxr[(mxr['DeathsR']>0)]


In [21]:
# Get data as occurred

#end_date    = date(2021, 2, 11)
end_date    = date(2020, 10, 5)
start_date  = date(2020, 4, 12)
break_date  = date(2020, 10, 6)
dateseries  = [date.fromordinal(i) for i in range(start_date.toordinal(), end_date.toordinal()+1)]

# Empty dataset
columns     = ['id','CLAVE_MUNICIPIO','CLAVE_ENTIDAD','DeathsO']
mxo         = pd.DataFrame(columns=columns)

for i in [end_date]:
    print('Working on: '+str(i), end="\r")
    
    # Extract year, month, day
    year = str(i.year-2000)
    if i.month<10:
        month = str(0)+str(i.month)
    else:
        month = str(i.month)
    if i.day<10:    
        day   = str(0)+str(i.day)
    else:   
        day   = str(i.day)

    # Download and extract file    
    filenamezip = 'datos_abiertos_covid19_'+day+'.'+month+'.20'+year+'.zip'
    if i.year<=2020:
        url = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/'+month+'/'+filenamezip
    else:
        url = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/20'+year+'/'+month+'/'+filenamezip
    filenamezip = wget.download(url)
    with zipfile.ZipFile(filenamezip, 'r') as zip_ref:
        zip_ref.extractall()

    # Read data and delete files
    filename = year+month+day+'COVID19MEXICO.csv'
    df = pd.read_csv(filename,encoding='latin1',low_memory=False)
    os.remove(filenamezip)
    os.remove(filename)

    # Municipality and entity
    #for j in range(0,len(xcodes)):
    for j in range(0,len(xcodes)):
        im = xcodes['CLAVE_MUNICIPIO'][j]
        ie = xcodes['CLAVE_ENTIDAD'][j]

        # Data selection
        if i<=break_date:
            s = df[
                (df['FECHA_DEF'] != '9999-99-99') & 
                (df['RESULTADO'] == 1) 
                &
                (df['MUNICIPIO_RES'] == im) &
                (df['ENTIDAD_RES'] == ie) 
            ][['FECHA_DEF','RESULTADO']] .groupby("FECHA_DEF") .count() .cumsum()
            s.index= pd.to_datetime(s.index)
            idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
            s = s.reindex(idx).fillna(method='ffill').fillna(0)
            s = s.rename(columns={'RESULTADO': 'DeathsO'})
            s['id'] = j
            s['CLAVE_MUNICIPIO'] = im
            s['CLAVE_ENTIDAD'] = ie
        else:
            s = df[
                (df['FECHA_DEF'] != '9999-99-99') & 
                (df['CLASIFICACION_FINAL'] <= 3) 
                &
                (df['MUNICIPIO_RES'] == im) &
                (df['ENTIDAD_RES'] == ie)
            ][['FECHA_DEF','CLASIFICACION_FINAL']] .groupby("FECHA_DEF") .count() .cumsum()
            s.index= pd.to_datetime(s.index)
            idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
            s = s.reindex(idx).fillna(method='ffill').fillna(0)
            s = s.rename(columns={'CLASIFICACION_FINAL': 'DeathsO'})
            s['id'] = j
            s['CLAVE_MUNICIPIO'] = im
            s['CLAVE_ENTIDAD'] = ie
        
        mxo = mxo.append(s)
        del s,idx
    del df

mxo         = mxo.reset_index()
mxo         = mxo.rename(columns={"index": "date"})    

In [22]:
# Save dataset 
mxo.to_csv('Data_Occurred_all_muni__mxo_nobreak.csv', index=False)

#pd.set_option('display.max_rows', 500)
#mxo[(mxo['CLAVE_MUNICIPIO']==15) & (mxr['CLAVE_ENTIDAD']==9)].tail()
#mxr[(mxr['DeathsR']>0)]

In [51]:
# Put both together
mxr = pd.read_csv('Data_Reported_all_muni__mxr.csv',encoding='latin1',low_memory=False)
mxr['date'] = mxr['date'].astype('datetime64[ns]')

mx = mxo.join(mxr[mxr['date']<='2020-10-05']['DeathsR']).reset_index()
#mx = mx.rename(columns={'index': 'Date'})
#mx['Diffpc']=(mx['DeathsO']-mx['DeathsR'])/mx['DeathsR']
mx['Delay']=0

In [52]:
#pd.set_option('display.max_rows', 500)
#mxr.head(150)
#mxr[mxr['date']<='20-10-05']['DeathsR'].head(150)
#mxr[mxr['date']<='2020-10-05'].head(50)
mx

,index,date,id,CLAVE_MUNICIPIO,CLAVE_ENTIDAD,DeathsO,DeathsR,Delay
0,0,2020-04-12,0,1,1,0.0,0.0,0
1,1,2020-04-13,0,1,1,2.0,0.0,0
2,2,2020-04-14,0,1,1,2.0,0.0,0
3,3,2020-04-15,0,1,1,2.0,0.0,0
4,4,2020-04-16,0,1,1,2.0,2.0,0
...,...,...,...,...,...,...,...,...
436300,436300,2020-10-01,2464,58,32,1.0,NaN,0
436301,436301,2020-10-02,2464,58,32,1.0,NaN,0
436302,436302,2020-10-03,2464,58,32,1.0,NaN,0
436303,436303,2020-10-04,2464,58,32,1.0,NaN,0


In [53]:
# Compute delays

for i in mx.index:
    if mx['DeathsR'][i] < mx['DeathsO'][i]:
        j=i
        while mx['DeathsR'][i] < mx['DeathsO'][j]:
            j=j-1
        if mx['id'][i]==mx['id'][j]:    
            mx.loc[i,'Delay'] = i-j-1 + (mx['DeathsO'][j+1]-mx['DeathsR'][i])/(mx['DeathsO'][j+1]-mx['DeathsO'][j])
        else:
            mx.loc[i,'Delay']=np.nan
            


In [54]:
mx.to_csv('Data_all_muni__mx_nobreak.csv', index=False)
#pd.set_option('display.max_rows', 500)
#mx[(mx['CLAVE_MUNICIPIO']==15) & (mx['CLAVE_ENTIDAD']==9)].tail()

In [109]:
pd.set_option('display.max_rows', 1500)
mx[mx['date']=='2020-04-12'].head(10)
#mx.head(1000)

,index,date,id,CLAVE_MUNICIPIO,CLAVE_ENTIDAD,DeathsO,DeathsR,Diffpc,Delay
0,0,2020-04-12,0,1,1,0.0,0.0,NaN,0.0
306,306,2020-04-12,1,2,1,0.0,0.0,NaN,0.0
612,612,2020-04-12,2,3,1,0.0,0.0,NaN,0.0
918,918,2020-04-12,3,4,1,0.0,0.0,NaN,0.0
1224,1224,2020-04-12,4,5,1,0.0,0.0,NaN,0.0
1530,1530,2020-04-12,5,6,1,0.0,0.0,NaN,0.0
1836,1836,2020-04-12,6,7,1,0.0,0.0,NaN,0.0
2142,2142,2020-04-12,7,8,1,0.0,0.0,NaN,0.0
2448,2448,2020-04-12,8,9,1,0.0,0.0,NaN,0.0
2754,2754,2020-04-12,9,10,1,0.0,0.0,NaN,0.0
